In [29]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# Machine learning packages
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV,LinearRegression

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## NEW CODE

In [30]:
# reading data from csv and creating a dataframe
df = pd.read_csv('data.csv')

# dataframe dimensions
print(f"This dataframe has {df.shape[0]} rows and {df.shape[1]} columns.")

This dataframe has 399284 rows and 33 columns.


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399284 entries, 0 to 399283
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            399284 non-null  int64  
 1   App Name              399283 non-null  object 
 2   App Id                399284 non-null  object 
 3   Category              399284 non-null  object 
 4   Rating                399284 non-null  float64
 5   Rating Count          399284 non-null  float64
 6   Installs              399284 non-null  float64
 7   Minimum Installs      399284 non-null  float64
 8   Maximum Installs      399284 non-null  int64  
 9   Free                  399284 non-null  bool   
 10  Price                 399284 non-null  float64
 11  Currency              399284 non-null  object 
 12  Size                  399284 non-null  float64
 13  Minimum Android       398998 non-null  object 
 14  Released              399284 non-null  object 
 15  

In [32]:
df = df.drop(['Unnamed: 0','Currency','Released','App Name','App Id','Category','Minimum Installs','Maximum Installs','Minimum Android','Last Updated','Content Rating','Ad Supported','In App Purchases','Editors Choice'], axis = 1)

In [33]:
X = df.drop(['Installs'],axis=1)
y = df['Installs']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.25,random_state = 345)


In [35]:
model = sm.OLS(y.astype(float), X.astype(float)).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:               Installs   R-squared (uncentered):                   0.986
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                          1.810e+06
Date:                Wed, 09 Dec 2020   Prob (F-statistic):                        0.00
Time:                        09:49:28   Log-Likelihood:                     -6.0674e+05
No. Observations:              399284   AIC:                                  1.214e+06
Df Residuals:                  399268   BIC:                                  1.214e+06
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [36]:
clf = LinearRegression()
clf.fit(X_train,y_train)
clf.predict(X_test)
clf.score(X_test,y_test)

0.8196789498864973

In [47]:
from numpy import arange
from sklearn.linear_model import LassoCV

lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(scale(X_train), y_train)
print("lasso coefficient",lassocv.alpha_)
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
print("Mean Square Error",mean_squared_error(y_test, lasso.predict(X_test)))
print("RSqaure",r2_score(y_test, lasso.predict(X_test)))


lasso coefficient 4.043814834122768e-06
Mean Square Error 1.120233910988469
RSqaure 0.8196789595226713


In [45]:
pd.Series(lasso.coef_, index=X.columns)


Rating                 -1.864903
Rating Count            0.948217
Free                    1.542148
Price                  -0.154600
Size                   -0.050247
c_lifestyle             0.105821
c_education             0.079508
c_health                0.052658
c_travel                0.000000
c_communication        -0.042355
c_news                  0.000000
c_tools                -0.102454
c_weather              -0.196616
c_business             -0.286747
c_entertainment        -0.075604
ReleaseMonth            0.001882
dayssincelastupdated   -0.000077
dayssincechange         0.000017
dtype: float64

In [13]:
lasso.score(X_test,y_test)

0.8196664537765167